# Optiver数据竞赛 EDA

先阅读了官方的指南：https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data


之后主要参考了分享的EDA：https://www.kaggle.com/chumajin/optiver-realized-eda-for-starter-english-version

# 一些概念

赛题提供了Order book和Trade的数据。
## Order Book
Order Book是呈现某种金融产品的买方（bid）和卖方（ask）意向价格的列表。如下图：


![order_book_1](https://www.optiver.com/wp-content/uploads/2021/05/OrderBook3.png)

## Trade
当买卖方价格一致时发生成交（Trade）。例如：在上图的市场下，出现买方想要买下20份股份，那么他会以148的价格成交，市场上标价148出售的股份相应会减少20份。


![order_book2](https://www.optiver.com/wp-content/uploads/2021/05/OrderBook4.png)

## 做市（Market making）和市场有效性（market efficiency）*
当A股票的Order book如下时，即ask和bid都比较稀疏时，可以说它的流通性比较低。
![order_book_3](https://www.optiver.com/wp-content/uploads/2021/05/OrderBook5.png)

这样的市场环境会比较差，举例：

''
You could insert an order to buy at 148. However, there is nobody currently willing to sell to you at 148, so your order will be sitting in the book, waiting for someone to trade against it. If you get unlucky, the price goes up, and others start bidding at 149, and you never get to buy at all. Alternatively, you could insert an order to buy at 155. The exchange would match this order against the outstanding sell order of one share at 149, so you buy 1 lot at 149. Similarly, you'd buy 12 shares at a price of 150, and 7 shares at 151. Compared to trying to buy at 148, there is no risk of not getting the trade that you wanted, but you do end up buying at a higher price.
''

Optiver的目的是维持市场的流通性，满足公众投资者的需求。即做市商（market maker）。
做市商是指在证券市场上，有具备一定实力和信誉的证券经营法人作为特许交易商，不断想公众投资者报出某些证券的买卖价格，并在该价位上接受公众投资者的买卖要求，以其自有资金和证券与投资者进行证券交易。

## Order book的数据特征
**bid/ask spread**

最好的bid和最好的ask之间的差距。
$$BidAskSpread = BestOffer/BestBid -1$$

**WAP, Weighted averaged price**

WAP，加权平均价格。

综合了Bid/Ask Price和Bid/Ask Size的价格。即综合了供求和实际价格的加权价格。
$$ WAP = \frac{BidPrice_{1}*AskSize_{1} + AskPrice_{1}*BidSize_{1}}{BidSize_{1} + AskSize_{1}} $$
注意，AskPrice > BidPrice。所以，当AskSize（供给量）增加时，WAP会增加；当BidSize（需求量）增加时，WAP降低。

**Log returns**

区别于百分比回报率的对数回报率。
$$
r_{t_1, t_2} = \log \left( \frac{S_{t_2}}{S_{t_1}} \right)
$$
性质：

- 可加性 例：$r_{t_1, t_2} + r_{t_2, t_3} = r_{t_1, t_3}$

- 无界性 百分比回报率不会小于-100%，对数回报率无下界。

## Realized volatility
预测目标，实际波动率，是实际价值（加权平均价格）的波动。

以WAP作为价格，求各个时间片的对数回报率（Log returns）$r_{t-1, t}$，然后求方差。

实际波动率为$\sigma$：
$$
\sigma = \sqrt{\sum_{t}r_{t-1, t}^2}
$$
方便起见，官方说明：并没有年化波动率；假设对数回报率均值为0。

# 数据介绍
加载比赛数据，观察分析。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. 预测目标

In [ ]:
# sample_submission.csv
sample = pd.read_csv("../input/optiver-realized-volatility-prediction/sample_submission.csv")
sample

## 2. 预测数据来源

In [ ]:
# test.csv
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test

提交样例中的row_id是stock_id和time_id用“-”连接得到的。

In [ ]:
# book_test.parquet
book_test_parquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0")
book_test_parquet

In [ ]:
# trade_test.parquet
trade_test_parquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0")
trade_test_parquet

目标为预测每支股票的波动率。

## 3. 训练数据

In [ ]:
# train.csv
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
train

target是股票在10分钟内的总波动率。

In [ ]:
#book_train.parquet
book_example = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0")
book_example 

book文件内容类似于买方和卖方的意向，但是由于双方意向的差距，并没有成交。
Bid是买方购买股票的意向价格，Ask是卖方出售股票的意向价格。（Ask > Bid）
- stock_id : 目前正在作为路径，之后可以考虑作为int类型的feature。
- time_id : 对应submission中的time_id
- seconds in bucket : 从0时刻开始之后的第多少秒？
- bid_price1,2: 最有吸引力的的bid的前两位
- ask_price1,2: 同上

In [ ]:
# trade_train.parquet
trade_example = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
trade_example

trade文件中为实际成交的状况，数据量会比book文件中稀疏。（实际成交的比改变意向的少很多）
- stock_id : 同上 
- time_id : 同上
- seconds_in_buckets : 同上。需要注意的是，由于trade数据的稀疏性，此处第一行的时间不一定是0.
- price ： 实际发生的成交在一秒内的平均价格。
- size ： 成交的股票份数。
- order_count : 发生不同的成交的个数。

# 数据分析

1. 按股票分析

In [ ]:
train

In [ ]:
for col in train.columns:
    print(col, ":", len(train[col].unique()))

共有112支股票，3830个不同的时间片.

target统计数据

In [ ]:
stock = train.groupby("stock_id")["target"].agg(["mean", "median", "std", "count", "sum"]).reset_index()
stock

In [ ]:
print("mean value=" ,stock["mean"].mean())
plt.hist(stock["mean"])

平均数为0.004，与0接近。

In [ ]:
print("sum value=" ,stock["sum"].mean())
plt.hist(stock["sum"])

这段时间波动率的叠加平均为14.8，有的超过了30.（波动率是可加的）

2. 同时间片下Bid， Ask以及实际产生的trade之间的关系

In [ ]:
book_example

In [ ]:
book_test = book_example[book_example["time_id"] == 5]
book_test


价格波动

In [ ]:
samples = ["bid_price1", "bid_price2", "ask_price1", "ask_price2"]

for num, a in enumerate(samples):
    plt.figure(figsize=(20,5))
    plt.subplot(4,1,num+1)
    plt.plot(book_test["seconds_in_bucket"], book_test[a])
    plt.title(a)
plt.show()
plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a)
plt.legend(fontsize=12)

In [ ]:
trade_example

In [ ]:
trade_test = trade_example[trade_example["time_id"]==5]
trade_test.head(5)

In [ ]:
plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    plt.plot(book_test["seconds_in_bucket"],book_test[a],label=a)
    
plt.scatter(trade_test["seconds_in_bucket"],trade_test["price"],label="trade_parquet",lw=7)
plt.legend(fontsize=12)

- 蓝色的点对应实际成交的时间以及价格。可以看到它在bid或者ask附近，偶尔会在线的中间。
- bid和ask在time_id=5的时间片（一共10min）也有很频繁的波动。
----

为了更好地理解波动率，我们分别找到波动率最小和最大的时间片。

In [ ]:
# min volatility time
stock0 = train[train["stock_id"]==0]
min_index = stock0["target"].idxmin()
min_time_id = stock0.iloc[min_index]["time_id"]
print("min index is",min_time_id,"and min target is",stock0.iloc[min_index]["target"])

In [ ]:
book_test_min = book_example[book_example["time_id"]==min_time_id]
trade_test_min = trade_example[trade_example["time_id"]==min_time_id]


plt.figure(figsize=(20,5))

for num,a in enumerate(samples): 
    plt.plot(book_test_min["seconds_in_bucket"],book_test_min[a],label=a)
    
plt.scatter(trade_test_min["seconds_in_bucket"],trade_test_min["price"],label="trade_parquet",lw=7)
plt.legend(fontsize=12)

In [ ]:
# max volatility time
stock0 = train[train["stock_id"]==0]
max_index = stock0["target"].idxmax()
max_time_id = stock0.iloc[max_index]["time_id"]
print("max index is",max_time_id,"and max target is",stock0.iloc[max_index]["target"])

In [ ]:
book_test_max = book_example[book_example["time_id"]==max_time_id]
trade_test_max = trade_example[trade_example["time_id"]==max_time_id]


plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    plt.plot(book_test_max["seconds_in_bucket"],book_test_max[a],label=a)
    
plt.scatter(trade_test_max["seconds_in_bucket"],trade_test_max["price"],label="trade_parquet",lw=7)
plt.legend(fontsize=12)

In [ ]:
plt.figure(figsize=(20,5))
plt.scatter(trade_test_min["seconds_in_bucket"],trade_test_min["price"],lw=10,label="min_vol_time")
plt.scatter(trade_test_max["seconds_in_bucket"],trade_test_max["price"],lw=10,label = "max_vol_time")
plt.legend(fontsize=15)

可以发现，放在同一尺度下比较的话，可以很明显的分辨波动率大小。

3. 特征工程

- price_spread : ask和bid的差距
- volumn: ask/bid的份数的多少
- volumn_imbalance : ask和bid之间的不平衡
- wap : 加权平均价格。即考虑供求关系后的价格。